# Mental Heatlh Kaggle Competition

## Imports and Config

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

In [2]:
plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [8]:
# Some useful paths

data_dir = Path("../data")
models_dir = Path("../models")
notebooks_dir = Path(".")

## Read data

In [29]:
data = pd.read_csv(data_dir / "train.csv")
data_to_predict = pd.read_csv(data_dir / "test.csv")
sample = pd.read_csv(data_dir / "sample_submission.csv")

In [33]:
data.isnull().sum()

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64

In [30]:
target = "Depression"
X, y = data.drop(columns=[target]), data[target]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

## EDA